### Percobaan 1: Matching untuk 3 Driver dan Customer
#### Distance & Time(dgn Openrouteservice), Order Count & Rating.
*) Referensi code dari project semester lalu.

In [1]:
import pandas as pd
import pyomo.environ as pyo
import requests as req
from dotenv import load_dotenv
from sklearn.preprocessing import minmax_scale
import os

In [2]:
# Melakukan loading data client dan driver dari CSV.
# File dummy dari project semester lalu.

clients = pd.read_csv('../datasets/data_client_small.csv')
drivers = pd.read_csv('../datasets/data_driver_small.csv')
banned = pd.read_csv('../datasets/banned_data_small.csv')

# SCALING
clients['rating_client'] = minmax_scale(clients['rating_client'])
drivers['rating_driver'] = minmax_scale(drivers['rating_driver'])

In [3]:
clients_data = clients.to_dict('records')
drivers_data = drivers.to_dict('records')
banned_data = banned.to_dict('records')
clients_data, drivers_data, banned_data

([{'id': 20,
   'latitude': -7.250317999983876,
   'longitude': 112.68898399990732,
   'rating_client': 1.0},
  {'id': 21,
   'latitude': -7.349470999997735,
   'longitude': 112.69903299988196,
   'rating_client': 0.0},
  {'id': 22,
   'latitude': -7.312413999992556,
   'longitude': 112.76892399970548,
   'rating_client': 0.75},
  {'id': 23,
   'latitude': -7.332430999995354,
   'longitude': 112.67340899994664,
   'rating_client': 0.75},
  {'id': 24,
   'latitude': -7.268699999986445,
   'longitude': 112.65469599999388,
   'rating_client': 0.25},
  {'id': 25,
   'latitude': -7.283033999988449,
   'longitude': 112.7348569997915,
   'rating_client': 1.0}],
 [{'id': 10,
   'latitude': -7.301529999991034,
   'longitude': 112.78152199967369,
   'rating_driver': 0.33333333333333337,
   'order_count': 78,
   'total_trip': 342.73199999997945,
   'time_idle': 683},
  {'id': 11,
   'latitude': -7.257838999984927,
   'longitude': 112.79037999965132,
   'rating_driver': 0.6666666666666666,
   'ord

In [4]:
# Load env untuk store API key
load_dotenv(override=True)

# Memo untuk tidak memanggil API untuk data yang sama.
memo = {}

In [6]:
from sklearn.preprocessing import minmax_scale

# Function untuk request ke ORS
def create_matrix(data, locations):
    durations_matrix, distances_matrix = {}, {}
    # SCALING
    data['durations'] = minmax_scale(data['durations'])
    data['distances'] = minmax_scale(data['distances'])
    for i, src in enumerate(locations):
        durations_matrix[src] = {}
        distances_matrix[src] = {}
        for j, dest in enumerate(locations):
            durations_matrix[src][dest] = data['durations'][i][j]
            distances_matrix[src][dest] = data['distances'][i][j]

    return distances_matrix, durations_matrix

def get_location_info(locations):
    data = memo.get(tuple(locations))

    if data != None:
        print("Cache hit")
        result = create_matrix(data, locations)
        memo[tuple(locations)] = data
        return result

    key = os.getenv('OPENROUTESERVICE_KEY')
    headers = {
        'Authorization': key,
        'Content-Type': 'application/json; charset=utf-8',
        'Accept': 'application/json, application/geo+json',
    }
    body = {
        'locations': [[i for i in locs] for locs in locations],
        'metrics': ['distance', 'duration'],
    }

    res = req.post(
        'https://api.openrouteservice.org/v2/matrix/driving-car',
        json=body,
        headers=headers
    )
    
    print(res.status_code)
    if res.status_code == 200:
        data = res.json()
        memo[tuple(locations)] = data
        return create_matrix(data, locations)
    else:
        return None

locations = []
for i in drivers_data:
    locations.append((i['longitude'], i['latitude']))
for i in clients_data:
    locations.append((i['longitude'], i['latitude']))

distances, durations = get_location_info(locations)

200


In [7]:
# Definisikan weight.
# Weight pada kondisi positif dan negatif, negatif berarti cenderung untuk lebih dipilih karena
#  menggunakan metode Minimize pada rumus.

# Tambahan weight pada rating, karena penalti akan jadi sangat kecil dibandingkan dengan distance dan duration.
weighted_rating = 10

weights = {
    'distance'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'duration'      : { 'target': 0, 'positive': 1, 'negative': -1 },
    'rating'        : { 'target': 0, 'positive': 1, 'negative': -1 },
}

In [8]:
# Karena data yang berpengaruh hanya data yang digabungkan antara keduanya, 
#   maka kami mengumpulkan data setelah dirumuskan dari masing-masing pasangan penumpang-driver
data: dict[tuple, dict[str, any]] = {}
for d in drivers_data:
    d_id = d['id']
    data[d_id] = {}
    for c in clients_data:
        c_id = c['id']
        distance = distances[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]
        duration = durations[(d['longitude'], d['latitude'])][(c['longitude'], c['latitude'])]
        rating = abs(d['rating_driver'] - c['rating_client'])

        data[d_id][c_id] = {
            "distance": distance,
            "duration": duration,
            "rating": rating,
        }

data

0.33333333333333337 1.0 0.6666666666666666
0.33333333333333337 0.0 0.33333333333333337
0.33333333333333337 0.75 0.41666666666666663
0.33333333333333337 0.75 0.41666666666666663
0.33333333333333337 0.25 0.08333333333333337
0.33333333333333337 1.0 0.6666666666666666
0.6666666666666666 1.0 0.33333333333333337
0.6666666666666666 0.0 0.6666666666666666
0.6666666666666666 0.75 0.08333333333333337
0.6666666666666666 0.75 0.08333333333333337
0.6666666666666666 0.25 0.41666666666666663
0.6666666666666666 1.0 0.33333333333333337
0.0 1.0 1.0
0.0 0.0 0.0
0.0 0.75 0.75
0.0 0.75 0.75
0.0 0.25 0.25
0.0 1.0 1.0
0.9999999999999999 1.0 1.1102230246251565e-16
0.9999999999999999 0.0 0.9999999999999999
0.9999999999999999 0.75 0.2499999999999999
0.9999999999999999 0.75 0.2499999999999999
0.9999999999999999 0.25 0.7499999999999999
0.9999999999999999 1.0 1.1102230246251565e-16
0.6666666666666666 1.0 0.33333333333333337
0.6666666666666666 0.0 0.6666666666666666
0.6666666666666666 0.75 0.08333333333333337
0.666

{10: {20: {'distance': 0.7093405692174476,
   'duration': 0.7849613216848736,
   'rating': 0.6666666666666666},
  21: {'distance': 0.643368979665982,
   'duration': 0.6462874612705987,
   'rating': 0.33333333333333337},
  22: {'distance': 0.1602256949021469,
   'duration': 0.23824505575994068,
   'rating': 0.41666666666666663},
  23: {'distance': 0.6746161433794187,
   'duration': 0.7363057975828501,
   'rating': 0.41666666666666663},
  24: {'distance': 0.6828679148696203,
   'duration': 0.7607579255281413,
   'rating': 0.08333333333333337},
  25: {'distance': 0.49620269678923895,
   'duration': 0.48142437279039063,
   'rating': 0.6666666666666666}},
 11: {20: {'distance': 0.6523360282861851,
   'duration': 0.7826116814888577,
   'rating': 0.33333333333333337},
  21: {'distance': 1.0, 'duration': 1.0, 'rating': 0.6666666666666666},
  22: {'distance': 0.5164174202279578,
   'duration': 0.6861820459470551,
   'rating': 0.08333333333333337},
  23: {'distance': 1.0, 'duration': 1.0, 'ratin

In [9]:
# Model pyomo utk solve GP
model = pyo.ConcreteModel()
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Variabel yang digunakan untuk mengidentifikasi driver dan customer adalah ID.
drivers = [d['id'] for d in drivers_data]
clients = [c['id'] for c in clients_data]

# Mendaftarkan semua kombinasi antara ID driver dan client.
model.x = pyo.Var(drivers, clients, domain=pyo.NonNegativeIntegers)

# Rumus untuk menghitung penalty negatif dan positif
def get_penalty(target, val, positive=True):
  if positive:
    if val > target: return val - target
    else: return 0
  else:
    if val < target: return target - val
    else: return 0

# Penalty = (weight positive * X * deviasi dgn target apabila >) + (weight negatif * X * deviasi dgn target apabila <)
# Salah satu dari deviasi ini akan menjadi 0 (target: 3, val: 5 maka sisi negatif akan 0)
def penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * model.x[d, c] * get_penalty(m['target'], data[d][c][param], False))
  )

# Hitung penalti dari matching yang sesuai (DV jadi 1.0)
# Ini dipakai di paling terakhir untuk menampilkan perhitungan penalti.
def v_penalty(c, d, param):
  m = weights[param]
  return (
    (m['positive'] * 1 * get_penalty(m['target'], data[d][c][param], True)) + 
    (m['negative'] * 1 * get_penalty(m['target'], data[d][c][param], False))
  )

# Objective function: minimalkan jumlah dari semua penalty masing-masing kolom data gabungan.
model.Cost = pyo.Objective(
    expr = sum([
        penalty(c, d, 'distance') + 
        penalty(c, d, 'duration') +
        penalty(c, d, 'rating')
        for c in clients for d in drivers
      ]),
    sense = pyo.minimize)

# Constraint yang digunakan yaitu pada jumlah nilai variabel pada setiap driver dengan semua kombinasi pasangan penumpangnya antara 0 atau 1 karena hanya memperbolehkan 1 driver mengambil 1 customer atau tidak mengambil customer sama sekali.
model.driver = pyo.ConstraintList()
for d in drivers:
  model.driver.add(sum(model.x[d, c] for c in clients) == 1)

model.client = pyo.ConstraintList()
for c in clients:
  model.client.add(sum(model.x[d, c] for d in drivers) == 1)

# Menambahkan constraint banned untuk pasangan penumpang dan driver yang tidak diperbolehkan ada di hasil matching.
model.banned = pyo.ConstraintList()
for i in banned_data:
  if i['client_id'] in clients and i['driver_id'] in drivers:
    model.banned.add(model.x[i['driver_id'], i['client_id']] == 0)

In [10]:
# Gunakan solver utk solve
result = pyo.SolverFactory('cbc').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5.24647703
  Upper bound: 5.24647703
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 35
  Number of nonzeros: 35
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of

In [11]:
# Menampilkan hasil matching dalam bentuk table (dari bawaan DataFrame)

print("Hasil dari Assignment Matching\n")
df = pd.DataFrame([], columns=clients, index=drivers)

for c in clients:
  mdr = []
  for d in drivers:
    mdr.append(model.x[d,c]())
  df[c] = mdr

df

Hasil dari Assignment Matching



,20,21,22,23,24,25
10,0.0,0.0,1.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,1.0
12,0.0,0.0,0.0,1.0,0.0,0.0
13,0.0,0.0,0.0,0.0,1.0,0.0
14,1.0,0.0,0.0,0.0,0.0,0.0
15,0.0,1.0,0.0,0.0,0.0,0.0


In [12]:
# Menampilkan hasil matching dan list penalti.
from pprint import pprint

result_dict = df.to_dict('dict')

for i in result_dict:
        for j in result_dict[i]:
                s = result_dict[i][j]
                if s == 1.0:
                        print(f"{j} ==> {i}")
print()

def find_pen(i):
        for j in clients:
                print(f"---- Penalties for Driver {i} -> Client {j}: ")
                distance_pen = v_penalty(j, i, 'distance')
                duration_pen = v_penalty(j, i, 'duration')
                rating_pen = v_penalty(j, i, 'rating')
                print("\tDistance:", distance_pen)
                print("\tDuration:", duration_pen)
                print("\tRating:", rating_pen)
                print("\tTotal penalty:", distance_pen + duration_pen + rating_pen)

for i in drivers:
        print(f"Driver {i}:")
        find_pen(i)
        for j in clients:
                if result_dict[j][i] == 1:
                        print(f"Match: Driver {i} -> Client {j}")
        print()


14 ==> 20
15 ==> 21
10 ==> 22
12 ==> 23
13 ==> 24
11 ==> 25

Driver 10:
---- Penalties for Driver 10 -> Client 20: 
	Distance: 0.7093405692174476
	Duration: 0.7849613216848736
	Rating: 0.6666666666666666
	Total penalty: 1.4943018909023214
---- Penalties for Driver 10 -> Client 21: 
	Distance: 0.643368979665982
	Duration: 0.6462874612705987
	Rating: 0.33333333333333337
	Total penalty: 1.2896564409365807
---- Penalties for Driver 10 -> Client 22: 
	Distance: 0.1602256949021469
	Duration: 0.23824505575994068
	Rating: 0.41666666666666663
	Total penalty: 0.3984707506620876
---- Penalties for Driver 10 -> Client 23: 
	Distance: 0.6746161433794187
	Duration: 0.7363057975828501
	Rating: 0.41666666666666663
	Total penalty: 1.4109219409622686
---- Penalties for Driver 10 -> Client 24: 
	Distance: 0.6828679148696203
	Duration: 0.7607579255281413
	Rating: 0.08333333333333337
	Total penalty: 1.4436258403977615
---- Penalties for Driver 10 -> Client 25: 
	Distance: 0.49620269678923895
	Duration: 0.4